In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
import uproot
import matplotlib as mpl
import math

from scipy import optimize
from scipy import interpolate, special
from scipy.stats import norm as normal

import cProfile

import inspect

import os

In [ ]:
# MPL config
mpl.rc('font', size=14)

dosave = False
savedir = "/home/grayputnam/Work/Winter2023/Mar18FitSignal/test/"
# override
if "SAVEDIR" in os.environ:
    savedir = os.environ["SAVEDIR"]

# Clear figures in main loop
CLEARPLT = False

In [ ]:
# Configuration

# Plane
IPLANE = 0
if "IPLANE" in os.environ:
    IPLANE = int(os.environ["IPLANE"])

# Default ER width
ER_WIDTH = 1.3
if "ER_WIDTH" in os.environ:
    ER_WIDTH = float(os.environ["ER_WIDTH"])
    
# Parameters for measurement resolution
if IPLANE == 2:
    # MC A
    SIGMA_A = 0.21
    SIGMA_B = 0.43
elif IPLANE == 1:
    SIGMA_A = 0.70
    SIGMA_B = 0.57
elif IPLANE == 0:
    SIGMA_A = 0.84
    SIGMA_B = 0.54
    
# Central index of electron path response
if IPLANE == 2:
    CENTER_IND = 625 # not banded
    # CENTER_IND = 610
elif IPLANE == 1:
    CENTER_IND = 608 # not banded
    # CENTER_IND = 595 # banded
elif IPLANE == 0:
    CENTER_IND = 591 # not banded
    # CENTER_IND = 578 # banded
    
    
FIT_PERIOD_HI = 40
FIT_PERIOD_LO = 15
PLT_PERIOD = 40

# Filename for WC configuration containing electron paths
# WCpaths_filename_nom = "./garfield-icarus-fnal-rev1.json"
# BANDED = False
#Sergey's WC paths
WCpaths_filename_nom = "./icarus_fnal_band_rev1.json"
BANDED = True

# Time spacing in file
time = np.linspace(0,100,1000, endpoint=False)

# Filename and histogram with electronics response
fER = "elecResp_ICARUS.root"
fERh = uproot.open(fER)["ER"]

# Filename with simulated signal response
SR_filename = "./mc/WFresults_Plane%i_WC.root" % IPLANE

angle_min = 20
angle_max = 80
angle_increment = 2
all_angles = np.array(list(range(angle_min, angle_max, angle_increment))) # 20-80, inclusive

# Whether to include Guassian broadening in fit
FITGAUS = False

# Include skew?
FITSKEW = False

N_CURRENT_TOY = 3000

# Plot various things
PLTPATHS = False # electron paths
PLTFR = False # field response
PLTSR = False # signal response
PLTFIT = False # fit

#Time increment
dt = time[1] - time[0]
dt

In [ ]:
# FIT_ANGLES = np.array([30, 36, 40, 46, 50, 54, 60, 64])
FIT_ANGLES = np.array(all_angles[0:-2])
FIT_ANGLES

In [ ]:
# Helper functions
def convolve(f1, f2):
    '''
    Return the simple convolution of the two arrays using FFT+mult+invFFT method.
    '''
    # fftconvolve adds an unwanted time shift
    #from scipy.signal import fftconvolve
    #return fftconvolve(field, elect, "same")
    s1 = np.fft.fft(f1)
    s2 = np.fft.fft(f2)
    sig = np.fft.ifft(s1*s2)

    return np.real(sig)

def gaus(t, sigma):
    return np.exp(-t**2/(2*sigma**2))

def skewgaus(t, sigma, skew):
    skew = -1/skew
    return gaus(t, sigma)*(1 + special.erf(skew*t/np.sqrt(2)))

def agaus(t, sigmalo, sigmahi):
    return np.exp(-t**2/(2*sigmalo**2))*(t < 0) + np.exp(-t**2/(2*sigmahi**2))*(t >= 0)

def norm_v(v):
    if IPLANE == 2:
        return v.max()
    else:
        return np.abs(v.min())

def norm(v, vnorm=None):
    if vnorm is None:
        vnorm = v
    return v / norm_v(vnorm)

def areanorm(v):
    return v / np.sum(v)
    
def center(t, v):
    if IPLANE == 0: # Ind-0 -- center on down-peak
        return t - t[np.argmin(v)]
    elif IPLANE == 2: # Collection -- center on peak
        return t - t[np.argmax(v)]
    else: # Ind-1 -- center on zero-cross
        center_ind = np.argmin(np.abs(v[np.argmax(v):np.argmin(v)])) + np.argmax(v)
        return t - t[center_ind]
    
    
def RC_filter(t, tau):
    return (t/tau-2)*(1/tau)*np.exp(-t/tau)*(t>=0)

def RC_tail(t, A, tau):
    return (A*dt/tau)*np.exp(-t/tau)*(t>=0)

In [ ]:
#Old electronics response lookup table
ER_val = fERh.values()
ER_time = fERh.axis().centers()
ER = interpolate.interp1d(ER_time, ER_val, kind="linear", bounds_error=False, fill_value=0)

#Parametrized electronics response
er_width = 1.3
def electronics_response_ideal(t,width):
    mu = 0
    amp = 10.
    y = (t>=mu)*(t<=10)*amp*(1-np.exp(-0.5*(0.9*(t-mu)/width)**2))*np.exp(-0.5*(0.5*(t-mu)/width)**2)
    return y

RC_FRAC = 0.161
RC_TAU = 51

def electronics_response_tail(t, width, frac=RC_FRAC, tau=RC_TAU):
    ER0 = electronics_response_ideal(t, width) 
    ER = convolve(ER0, RC_tail(t, frac, tau)) + ER0
    return ER

In [ ]:
plt.clf()

plt.plot(time, electronics_response_ideal(time,er_width) / electronics_response_ideal(time,er_width).max())
plt.xlabel("Time [$\\mu$s]")
plt.ylabel("Amplitude")
plt.xlim([0, 10])
plt.title("Ideal Electronics Response")
plt.tight_layout()

if dosave: plt.savefig(savedir + "elecResp.pdf")

In [ ]:
plt.clf()

ER_plt = electronics_response_ideal(time,er_width)
plt.plot(time, ER_plt / ER_plt.max(), label="Bessel")

ER_plt_conv = -convolve(ER_plt, RC_filter(time, 1)) + ER_plt
plt.plot(time, ER_plt_conv / np.abs(ER_plt_conv).max(), label="Bessel $\\circledast$ RC-RC")
plt.xlim([0, 15])
plt.legend()

In [ ]:
plt.clf()

ER_plt = electronics_response_ideal(time, er_width)
plt.plot(time, ER_plt, label="Bessel")

ER_plt_conv = electronics_response_tail(time, er_width, frac=-0.2, tau=4)
plt.plot(time, ER_plt_conv, label="Bessel $\\circledast$ RC")
plt.xlim([0, 15])
#plt.yscale("log")
#plt.ylim([1e-4, 1])
plt.legend()

# plt.plot(time, RC_tail(time, 0.133, 65)*(0.1/65)/np.sum(ER_plt))

In [ ]:
ER_time_plt = np.linspace(0, 1_000, 10_000)

In [ ]:
plt.clf()

ER_plt = electronics_response_ideal(ER_time_plt, er_width)
plt.plot(ER_time_plt, ER_plt, label="Bessel")

ER_plt_conv = electronics_response_tail(ER_time_plt, er_width)
plt.plot(ER_time_plt, ER_plt_conv, label="Bessel $\\circledast$ RC-tail")
#plt.xlim([0, 15])
plt.yscale("log")
plt.ylim([1e-4, 10])
plt.legend()
plt.xlim([0, 200])

# plt.plot(time, RC_tail(time, 0.133, 65)*(0.1/65)/np.sum(ER_plt))

In [ ]:
dat = json.load(open(WCpaths_filename_nom))

driftV = dat["FieldResponse"]["speed"]*1e3 # mm/us

thispaths = dat["FieldResponse"]["planes"][IPLANE]["PlaneResponse"]["paths"]

pitchpos_f = [path["PathResponse"]["pitchpos"] for path in thispaths] # mm

paths_f = [np.array(path["PathResponse"]["current"]["array"]["elements"])
             for path in thispaths]

pitchpos = []
paths_nom = []

for i, (ppos, path) in enumerate(zip(pitchpos_f, paths_f)):
    if BANDED:
        if ppos == 0:
            path = path *11/12
        elif ppos == -1.5:
            path = path*11/6
        
    pitchpos.append(ppos)
    paths_nom.append(path)
    if -ppos not in pitchpos_f:
        pitchpos.append(-ppos)
        paths_nom.append(path)
    # handle boundary between (half-)wires
    else:
        paths_nom[-1] = tuple([paths_nom[-1], paths_f[pitchpos_f.index(-ppos)]])
        
pitchpos, paths_nom = zip(*sorted(zip(pitchpos, paths_nom), key=lambda pair: pair[0]))
pitchpos = np.array(pitchpos)

pitchpos_interp = np.linspace(pitchpos.min(), pitchpos.max(), 2101) + 0.015
driftV

In [ ]:
plt.clf()

for p, path in zip(pitchpos, paths_nom):
    if isinstance(path, tuple):
        plt.plot(time, -path[0])
    else:
        plt.plot(time, -path)

plt.title("Wire-Cell Electron Path Responses")
plt.xlabel("Time [$\\mu$s]")
plt.ylabel("Current")

plt.xlim([57 - 2*(2-IPLANE) - 5, 67 - 2*(2-IPLANE) + 5])
plt.tight_layout()

# plt.axvline([time[CENTER_IND]], color="r")

if dosave: plt.savefig(savedir + "pathResp.pdf", bbox_inches="tight")

In [ ]:
txt_x = 0.625

In [ ]:
# Pick plane, load in data, etc

# Filename with measured signal response
if IPLANE == 0:
    file_path = "./data/RespStudiesData_Apr2023_RightBaseline/"
    BASELINE = "Right"
else:
    file_path = "./data/RespStudiesData_Apr2023_LeftBaseline/"
    BASELINE = "Left"

data_files = []
run_list = os.listdir(file_path)
tpc_list = ["EE","EW","WE","WW"]
for run in run_list:
    for tpc in tpc_list:
        data_files.append(uproot.open(file_path+run+"/WFresults_Plane"+str(IPLANE)+"_"+tpc+".root"))
        
# data_files = [uproot.open("./mc/far_TrackBased_official/WFresults_Plane%i_XX.root" % IPLANE)]
        
#Create arrays over angle bins to store angle info, data filenames, data products
angle_min = 20
angle_max = 80
angle_increment = 2

nall_angles = len(all_angles)
nfiles = len(data_files)
ndata = 401
data_hists    = np.zeros((nall_angles, ndata))
data_err_vecs = np.zeros((nall_angles, ndata))

data_times = []
data_when_list = []
data_fit_list = []
data_which_fit = []

for j,angle in enumerate(all_angles):
    thlo = angle
    thhi = thlo + angle_increment
    
    FIT_PERIOD = FIT_PERIOD_LO if thlo < 40 else FIT_PERIOD_HI
    
    angle_data_hists = []
    angle_err_hists = []
    
    set_times = False
    for i, uhf in enumerate(data_files):
        data = uhf["AnodeRecoHist1D_%ito%i" % (thlo, thhi)]
        data_err = uhf["AnodeTrackUncertHist2D_%ito%i" % (thlo, thhi)].to_numpy()[0][5]
        
        # ignore cases where the error is too low (no tracks)
        if (data_err < 1e-4).any(): continue
        
        angle_data_hists.append(data.values())
        angle_err_hists.append(data_err)
        
        if not set_times:
            data_times.append(center(data.axis().centers(), data.values()))
            data_when_list.append(np.abs(data_times[-1]) < PLT_PERIOD)
            data_fit_list.append(np.abs(data_times[-1]) < FIT_PERIOD)
            data_which_fit.append( data_fit_list[-1][data_when_list[-1]] )
            set_times = True

    angle_data_hists = np.array(angle_data_hists)
    angle_err_hists = np.array(angle_err_hists)
    
    # Average together weighted by uncertainty
    data_hists[j][:] = np.sum(angle_data_hists / angle_err_hists**2, axis=0) / np.sum(1 / angle_err_hists**2, axis=0)
    
    # Use std-dev to get uncertainty -- ignore intrinsic error
    # NOTE: we end up not really using this anyway
    data_err_vecs[j][:] = np.std(angle_data_hists, axis=0) / len(data_files)
    
data_fit_all = np.concatenate(data_which_fit[0:-2])

In [ ]:
toy_time = np.linspace(0, N_CURRENT_TOY*0.1, N_CURRENT_TOY, endpoint=False)

In [ ]:
#Helper function to compute chi2
def calc_chi2(pred, meas, err):
    return np.sum(((meas - pred)/err)**2)

#Convert from prepared time (containing angle info) to actual time
def convert_time(t0):
    angle_bin = np.floor(t0/(2*PLT_PERIOD)).astype(int)
    angle_bin_set = to_set(angle_bin)
    angle_bin_index = np.zeros(len(angle_bin),dtype=int)
    for i in range(len(angle_bin_set)):
        angle_bin_index += i*(angle_bin==angle_bin_set[i]).astype(int)
    t = (t0 % (2*PLT_PERIOD)) - PLT_PERIOD
    angle = (all_angles[angle_bin_set]+all_angles[angle_bin_set+1])/2
    
    return (t,angle_bin_set,angle_bin_index,angle)

def contains(array,x):
    return np.any(array == x)

#returns the set of elements in the input array - no duplicates
def to_set(array):
    return np.unique(array)

#Takes as input the scaled times for each path, the WC paths and the interpolated path positions and returns a list of interpolated paths
def interpolate_paths(paths, pitchpos_interp):
    paths_interp = []

    for j, p in enumerate(pitchpos_interp):
        i_pitchpos = int((p - pitchpos[0]+1e-6) / (pitchpos[1] - pitchpos[0]))
        if i_pitchpos == len(pitchpos) - 1:
            path = paths[i_pitchpos]
        else:
            F1 = paths[i_pitchpos]
            F2 = paths[i_pitchpos+1]

            # handle boundary between (half-)wires
            if isinstance(F1, tuple):
                if p > pitchpos[i_pitchpos]:
                    F1 = F1[0]
                else:
                    F1 = F1[1]
            if isinstance(F2, tuple):
                if p > pitchpos[i_pitchpos+1]:
                    F2 = F2[0]
                else:
                    F2 = F2[1]

            interp = (pitchpos[i_pitchpos+1] - p)/(pitchpos[1] - pitchpos[0])
            path = F1*interp + F2*(1-interp)

        paths_interp.append(path)
    return paths_interp

def shifted_paths_sum(angle, paths_interp, pitchpos_interp):
    n_current_path = paths_interp[0].size
    t0_shift = (N_CURRENT_TOY - n_current_path) // 2
    
    thxw = angle*np.pi/180
    shift = (np.outer(np.tan(thxw), pitchpos_interp/driftV/dt)).astype(int)
    summed_paths_angle = []
    
    for i in range(len(angle)):
        summed_paths_angle.append(np.zeros(N_CURRENT_TOY))
        
        for j in range(len(paths_interp)):
            s = shift[i][j]
            
            # ignore if there is nothing to set
            if t0_shift+s > N_CURRENT_TOY or t0_shift+s+n_current_path < 0:
                continue
            
            set_tl0 = max(0, t0_shift+s)
            set_thi = min(N_CURRENT_TOY, t0_shift+s+n_current_path)
            
            get_tlo = max(0, -t0_shift-s)
            get_thi = min(n_current_path, (N_CURRENT_TOY - s - n_current_path))
            
            summed_paths_angle[-1][set_tl0:set_thi] += -paths_interp[j][get_tlo:get_thi]
                
    return summed_paths_angle

#Finds the index where a strictly increasing array first goes above 0
def find_zero(array):
    return np.argmax(array >= 0)

#Scale the array of input times t by values sl and sr for the left and right sides of 0, respectively
def scale_time(t,sl,sr, sl2, sltau2, sr2, srtau2, sl4, sltau4, sr4, srtau4):
    t0_index = find_zero(t)
    scaled_time = np.zeros(len(t))
    scaled_time[:t0_index] = t[:t0_index]*(sl + sl2/(1 + (t[:t0_index]/sltau2)**2) + sl4/(1 + (t[:t0_index]/sltau4)**4))
    scaled_time[t0_index:] = t[t0_index:]*(sr + sr2/(1 + (t[t0_index:]/srtau2)**2) + sr4/(1 + (t[t0_index:]/srtau4)**4))
    return scaled_time

#Weights each path based on left and right parameters wl, wr
def weight_path(path, path_time, wl, wr, expl, expl_start, expr, expr_start, tal, tar, talexp, tarexp, al1, al2, al3, ar1, ar2, ar3):
    center_index = find_zero(path_time)
    path_left  = path*wl*(1 + tal*np.abs((path_time - path_time[center_index]))**talexp)
    path_left *= np.exp((np.abs(path_time - path_time[center_index])>expl_start)*expl*np.abs((path_time - path_time[center_index])))
    path_left = al1*path_left + al3*path_left**2*np.sign(path_left) + al3*path_left**3
    path_right = path*wr*(1 + tar*np.abs((path_time - path_time[center_index]))**tarexp)
    path_right *= np.exp((np.abs(path_time - path_time[center_index])>expr_start)*expr*np.abs((path_time - path_time[center_index])))
    path_right = ar1*path_right + ar2*path_right**2*np.sign(path_right) + ar3*path_right**3
    
    return np.concatenate((path_left[:center_index], path_right[center_index:]))

#Computes a scaled path at given offset by evaluating an input path at times path_time
def scale_path(path, path_time, 
               scale_left, scale_right, 
               scale_left2, sltau2, scale_right2, srtau2,
               scale_left4, sltau4, scale_right4, srtau4,
               al1, al2, al3, ar1, ar2, ar3,
               wl, wr, expl, expl_start, expr, expr_start, tal, tar, talexp, tarexp, time_shift):
    
    scaled_time = scale_time(path_time + time_shift, 
                             scale_left, scale_right, 
                             scale_left2, sltau2, scale_right2, srtau2,
                             scale_left4, sltau4, scale_right4, sltau4)
    
    if isinstance(path, tuple):
        weighted_path_left  = weight_path(path[0],path_time,wl,wr, expl, expl_start, expr, expr_start, tal,tar, talexp, tarexp, al1, al2, al3, ar1, ar2, ar3)
        weighted_path_right = weight_path(path[1],path_time,wl,wr, expl, expl_start, expr, expr_start, tal,tar, talexp, tarexp, al1, al2, al3, ar1, ar2, ar3)
        pathl_interp = interpolate.interp1d(path_time, weighted_path_left, kind="linear", bounds_error=False, fill_value=0)
        pathr_interp = interpolate.interp1d(path_time, weighted_path_right, kind="linear", bounds_error=False, fill_value=0)
        scaled_path = [pathl_interp(scaled_time), pathr_interp(scaled_time)]
        scaled_path = tuple(scaled_path)

    else:
        weighted_path = weight_path(path,path_time,wl,wr, expl, expl_start, expr, expr_start, tal,tar, talexp, tarexp, al1, al2, al3, ar1, ar2, ar3)
        path_interp = interpolate.interp1d(path_time, weighted_path, kind="linear", bounds_error=False, fill_value=0)
        scaled_path = path_interp(scaled_time)  
        
    return scaled_path

#Sum the field responses over drift offset to get the overall field response
def compute_field_response(shifted_paths, angles):
    field_response_angles = []
    for i in range(nangles):
        field_response_angles.append(-sum(shifted_paths[i]))
    return field_response_angles

# TODO FIX -- this doesn't really work
def compute_path_center_ind(path):
    if isinstance(path, tuple):
        v = -path[0]
    else:
        v = -path
        
    if IPLANE == 2:
        center_ind = np.argmax(v)
    #elif IPLANE == 0:
    #    center_ind = np.argmin(v)
    else: # Ind-1 -- center on zero-cross
        center_ind = np.argmax(v[575:] <= 0) + 575
    return center_ind

def compute_path_center_time(time):
    return time - time[CENTER_IND]

In [ ]:
centers = []
for p in paths_nom[100:110]:
    centers.append(compute_path_center_ind(p))
centers

In [ ]:
def mc_waveform(t0, er_width=1.3, rcrc_width=0, rc_neg_width=0, rc_neg_A=0,
                sl0=1, sr0=1, sl1=0, sr1=0, sl2=0, sr2=0,
                sl02=0, sr02=0, sl12=0, sr12=0, sl22=0, sr22=0,
                sl04=0, sr04=0, sl14=0, sr14=0, sl24=0, sr24=0,
                sltau02=1, srtau02=1, sltau04=1, srtau04=1,
                sltau12=0, srtau12=0, sltau14=0, srtau14=0,
                weight_ratio=0, shiftc=0,
                tal=0, tar=0, talexp=1, tarexp=1, 
                expl=0, expl_start=1, expr=0, expr_start=1,
                al01=1, al02=0, al03=0, ar01=1, ar02=0, ar03=0,
                al11=0, al12=0, al13=0, ar11=0, ar12=0, ar13=0,
                polyr2=1, polyr2const=0, polyr2len=3, polyr2exp=1, polyr2start=0,
                polyl2=1, polyl2const=0, polyl2len=3, polyl2exp=1, polyl2start=0,
                sigma_a=SIGMA_A, sigma_b=SIGMA_B, mu0=[0],
                BASELINE=None, ANGLES=None, normalize=True, include_tail=True,
                N_ANGLE_SPACING=11):
    
    if ANGLES is None:
        ANGLES = MC_ANGLES + 1
        
    assert(BASELINE is None or BASELINE == "Left" or BASELINE == "Right")
    
    nangle = len(ANGLES)
    if len(mu0) == 1:
        mu0 = mu0*nangle
    else: 
        assert(len(mu0) == nangle)
        
    t0_angle = np.split(t0, nangle)
    
    #Compuate scaled times and interpolate to get scaled paths, all for each angle
    offset_param = 1-np.exp(-np.absolute(pitchpos)/1.5)  #3 mm between wires
    # offset_param = (np.abs(pitchpos)%3)/3
    scaling_left  = sl1*offset_param + sl2*offset_param**2
    scaling_right = sr1*offset_param + sr2*offset_param**2
    
    scaling_left2  = sl12*offset_param + sl22*offset_param**2
    scaling_right2 = sr12*offset_param + sr22*offset_param**2
    
    scaling_left4  = sl14*offset_param + sl24*offset_param**2
    scaling_right4 = sr14*offset_param + sr24*offset_param**2
    
    sltau2 = sltau02 + sltau12*offset_param
    srtau2 = srtau02 + srtau12*offset_param
    sltau4 = sltau04 + sltau14*offset_param
    srtau4 = srtau04 + srtau14*offset_param
    
    al1 = al01 + offset_param*al11
    al2 = al02 + offset_param*al12
    al3 = al03 + offset_param*al13
    ar1 = ar01 + offset_param*ar11
    ar2 = ar02 + offset_param*ar12
    ar3 = ar03 + offset_param*ar13
    
    weights_left  = (1 - weight_ratio)
    weights_right = 1./(1 - weight_ratio)
    
    time_shift = offset_param*shiftc
    
    scaled_paths = []
    for i, p_nom in enumerate(paths_nom):
        t = compute_path_center_time(time)
        #dp = (a1*t + a2*t**2 + a3*t**3 + a4*t**4 + a5*t**5) * np.exp(-atau*np.abs(t))
        #dp = np.exp(-np.abs(pitchpos[i])/polyr2len)*1e-4*polyr2const/(1 + polyr2*np.abs(t)**polyr2exp*(t>=polyr2start))
        #dp+= np.exp(-np.abs(pitchpos[i])/polyl2len)*1e-4*polyl2const/(1 + polyl2*np.abs(t)**polyl2exp*(t<=polyl2start))
        dp = np.exp(-np.abs(pitchpos[i])/polyr2len)*1e-4*polyr2const/(1+polyr2*np.abs(t)**polyr2exp)*(t>=polyr2start)
        dp+= np.exp(-np.abs(pitchpos[i])/polyl2len)*1e-4*polyl2const/(1+polyl2*np.abs(t)**polyl2exp)*(t>=polyl2start)
        
        p = p_nom
        if isinstance(p_nom, tuple):
            if np.abs(pitchpos[i]) <= 1.5:
                p = (p[0] + dp, p[1] + dp)
        else:
            if np.abs(pitchpos[i]) <= 1.5:
                p += dp
                        
        
        sp = scale_path(p, t,
                        sl0 + scaling_left[i], sr0 + scaling_right[i],
                        sl02 + scaling_left2[i], sltau2[i], sr02 + scaling_right2[i], srtau2[i],
                        sl04 + scaling_left4[i], sltau4[i], sr04 + scaling_right4[i], srtau4[i],
                        al1[i], al2[i]*1e3, al3[i]*1e6, ar1[i], ar2[i]*1e3, ar3[i]*1e6,
                        weights_left, weights_right, expl, expl_start, expr, expr_start, tal, tar, talexp, tarexp, time_shift[i])
        scaled_paths.append(sp)
            
    # interpolate the paths with a finer spacing (0.03mm)
    # avoid edge effects by spacing in between the discontinuities in the paths (every 1.5mm)
    paths_interp = interpolate_paths(scaled_paths, pitchpos_interp)
        
    # Compute the interpolated field response at each track angle
    field_response_angles = []
    for a in ANGLES:
        if a >= 70: # only important for high angles
            sum_angles = np.linspace(a-1, a+1, N_ANGLE_SPACING, endpoint=True)
            this_fr = np.average(shifted_paths_sum(sum_angles, paths_interp, pitchpos_interp), axis=0)
        else:
            this_fr = shifted_paths_sum(np.array([a]), paths_interp, pitchpos_interp)[0]
        
        field_response_angles.append(this_fr)
        
    #Convolve with electronics response and Gaussian smearing
    gaus_sigma = np.sqrt(sigma_a**2 + (np.tan(ANGLES*np.pi/180)*sigma_b)**2)
    
    # Build the electronics response
    ER = electronics_response_tail(toy_time, er_width) if include_tail else electronics_response_ideal(toy_time, er_width)
    if rcrc_width > 0:
        ER = -convolve(ER, RC_filter(toy_time, rcrc_width)) + ER
    if rc_neg_width > 0:
        ER = -convolve(ER, RC_tail(toy_time, rc_neg_A, rc_neg_width)) + ER
    if normalize:
        ER = ER / np.abs(ER).max()
    
    mc_val_interp_list = []
    vals = []
    for i in range(nangle):
        if er_width > 0:
            SR = convolve(field_response_angles[i], ER)
        else:
            SR = field_response_angles[i]
        if gaus_sigma[i] > 1e-4:
            SR_gaus = convolve(SR, gaus(toy_time, gaus_sigma[i]))
        else:
            SR_gaus = SR
        if normalize:
            SR_gaus = norm(SR_gaus)
        centered_time = center(toy_time, SR_gaus)
        SR_interp = interpolate.interp1d(centered_time + mu0[i], SR_gaus, kind="linear", bounds_error=False, fill_value=0)
        
        signal = SR_interp(t0_angle[i])
        
        # Fix the baseline
        if BASELINE == "Left":
            baseline_times = np.linspace(-80, -64, int(16/0.4)+1)
            signal -= np.average(SR_interp(baseline_times))
        elif BASELINE == "Right":
            baseline_times = np.linspace(64, 80, int(16/0.4)+1)
            signal -= np.average(SR_interp(baseline_times))
            
        if normalize:
            signal = norm(signal)
        
        vals.append(signal)

    return np.concatenate(vals)

In [ ]:
FIT_ANGLES

In [ ]:
#Create arrays for time,data in fit and plot regions
time_fitregion_list = []
time_plotregion_list = []

data_fitregion_list = []
data_plotregion_list = []
data_fitregion_norm_list = []
data_plotregion_norm_list = []

data_err_fitregion_list = []
data_err_plotregion_list = []
data_err_fitregion_norm_list = []
data_err_plotregion_norm_list = []

#Loop over angles
nangles = len(FIT_ANGLES)
for i_angle in range(nangles):
    angle = FIT_ANGLES[i_angle]
    i_angle_h = np.where(all_angles == angle)[0][0]
    thlo = angle
    thhi = thlo + angle_increment
    
    time_fitregion_list.append(data_times[i_angle_h][data_when_list[i_angle_h]])
    time_plotregion_list.append(data_times[i_angle_h][data_when_list[i_angle_h]])

    data_fitregion_list.append(data_hists[i_angle_h][data_when_list[i_angle_h]])
    data_plotregion_list.append(data_hists[i_angle_h][data_when_list[i_angle_h]])
    data_fitregion_norm_list.append(norm(data_fitregion_list[-1]))
    data_plotregion_norm_list.append(norm(data_plotregion_list[-1]))

    data_err_fitregion_list.append(data_err_vecs[i_angle_h][data_when_list[i_angle_h]])
    data_err_plotregion_list.append(data_err_vecs[i_angle_h][data_when_list[i_angle_h]])
    data_err_fitregion_norm_list.append(norm(data_err_fitregion_list[-1], data_fitregion_list[-1]))
    data_err_plotregion_norm_list.append(norm(data_err_plotregion_list[-1], data_plotregion_list[-1]))

time_fitregion           = np.concatenate(time_fitregion_list)
time_plotregion          = np.concatenate(time_plotregion_list)

data_fitregion           = np.concatenate(data_fitregion_list)
data_plotregion          = np.concatenate(data_plotregion_list)
data_fitregion_norm      = np.concatenate(data_fitregion_norm_list)
data_plotregion_norm     = np.concatenate(data_plotregion_norm_list)

data_err_fitregion       = np.concatenate(data_err_fitregion_list)
data_err_plotregion      = np.concatenate(data_err_plotregion_list)
data_err_fitregion_norm  = np.concatenate(data_err_fitregion_norm_list)
data_err_plotregion_norm = np.concatenate(data_err_plotregion_norm_list)

# Fit

# everything you could fit for -- using the mc_waveform spec
all_fit_params = inspect.getfullargspec(mc_waveform).args[1:-5]

# all defaults and bounds
default_params = dict(zip(all_fit_params, inspect.getfullargspec(mc_waveform).defaults))

# These parameters have a bound at their nominal value. This creates 
# issues with the fit, so instead set the default to a (not-nominal)
# central value in the bound
default_params["rcrc_width"] = 1
default_params["rc_neg_width"] = 1
default_params["rc_neg_A"] = 0.1

default_params["expl"] = -0.005
default_params["expr"] = -0.005

# setup bounds manually
bounds = {
    "er_width": (0.5, 1.5),
    "rcrc_width": (0, 2),
    "rc_neg_width": (0, 4),
    "rc_neg_A": (0, 1),
    
    "sl0": (0.05, 5),
    "sr0": (0.05, 5),
    "sl1": (-0.2, 2),
    "sr1": (-0.2, 2),
    "sl2": (-0.2, 2),
    "sr2": (-0.2, 2),
    
    "sl02": (-1, 5),
    "sr02": (-1, 5),
    "sl12": (-0.2, 2),
    "sr12": (-0.2, 2),
    "sl22": (-0.2, 2),
    "sr22": (-0.2, 2),
    
    "sl04": (-1, 5),
    "sr04": (-1, 5),
    "sl14": (-0.2, 2),
    "sr14": (-0.2, 2),
    "sl24": (-0.2, 2),
    "sr24": (-0.2, 2),
    
    "sltau02": (0, 20),
    "srtau02": (0, 20),
    "sltau04": (0, 20),
    "srtau04": (0, 20),
    
    "sltau12": (-5, 5),
    "srtau12": (-5, 5),
    "sltau14": (-5, 5),
    "srtau14": (-5, 5),

    "al01": (0.5, 1.5),
    "al02": (-10, 10),
    "al03": (-10, 10),
    "al11": (-1, 1),
    "al12": (-10, 10),
    "al13": (-10, 10),
    
    "ar01": (0.5, 1.5),
    "ar02": (-10, 10),
    "ar03": (-10, 10),
    "ar11": (-1, 1),
    "ar12": (-10, 10),
    "ar13": (-10, 10),
    
    "tal": (-1, 1),
    "tar": (-1, 1),
    "talexp": (0, 5),
    "tarexp": (0, 5),
    "expl": (-0.05, 0),
    "expr": (-0.05, 0),
    "expl_start": (0, 30),
    "expr_start": (0, 30),

    "polyl2": (0, 10),
    "polyl2const": (-1, 1),
    "polyl2exp": (0, 5),
    "polyl2len": (0, 30),
    "polyl2start": (-1, 20),
    
    "polyr2": (0, 10),
    "polyr2const": (-10, 10),
    "polyr2exp": (0, 5),
    "polyr2start": (-1, 20),
    "polyr2len": (0, 30),
    

    "weight_ratio": (-0.9, 0.9),
    "shiftc": (-2, 2),
    "sigma_a" : (SIGMA_A / 2, SIGMA_A * 2),
    "sigma_b" : (SIGMA_B / 2, SIGMA_B * 2),
    "mu0": (-2, 2)
}

assert(default_params.keys() == bounds.keys())

nmu = nangles
# nmu = 1

# Option to fit for each waveform center individually
for i in range(nmu):
    name = "mu" + str(i)
    default_params[name] = 0
    bounds[name] = (-2, 2)
    if i > 0:
        all_fit_params.append(name)
        
MC_ANGLES = FIT_ANGLES
MC_ANGLES

In [ ]:
debugging = True

THIS_ANGLES = np.array([20, 50, 70, 74])

#Default prediction for comparison
do_fit = False

thistimes = np.linspace(-80, 80, 160*2 + 160//2, endpoint=False)
thistimes = np.linspace(-40, 40, 80*2 + 80//2, endpoint=False)
thistimes_many = np.concatenate([thistimes]*len(THIS_ANGLES))

pred_default_plotregion = mc_waveform(thistimes_many, N_ANGLE_SPACING=11, BASELINE=BASELINE, ANGLES=THIS_ANGLES+1)
pred_default_plotregion_norm_list = [norm(v) for v in np.split(pred_default_plotregion, len(THIS_ANGLES))]

pred_default_plotregion_weird = mc_waveform(thistimes_many, 
                                            N_ANGLE_SPACING=1,
                                            BASELINE=BASELINE, ANGLES=THIS_ANGLES+1)
pred_default_plotregion_norm_weird_list = [norm(v) for v in np.split(pred_default_plotregion_weird, len(THIS_ANGLES))]

plt.close("all")

for i,a in enumerate(THIS_ANGLES):
    thlo = a
    thhi = thlo + angle_increment
    
    plt.figure(i)
    #plt.errorbar(time_plotregion_list[i], data_plotregion_norm_list[i], data_err_plotregion_norm_list[i],
    #         linestyle="none", marker=".", color="black", label="Data")
    
    plt.plot(thistimes, pred_default_plotregion_norm_list[i])
    plt.plot(thistimes, pred_default_plotregion_norm_weird_list[i])
    plt.text(txt_x, 0.8, "$%i^\\circ < \\theta_{xw} < %i^\\circ$" % (thlo, thhi),
             fontsize=16,transform=plt.gca().transAxes)

In [ ]:
# Define the parameters we will fit for here

tofit = [
    "er_width",
    "rcrc_width",
    
    "sl0",
    "sr0",
    "sl1",
    "sr1",
    
    "sl02",
    "sr02",
    "sl12",
    "sr12",
    
    "sl04",
    "sr04",
    "sl14",
    "sr14",
    
    "sltau02",
    "srtau02",
    "sltau04",
    "srtau04",
    "sltau12",
    "srtau12",
    "sltau14",
    "srtau14",

    "al01",
    "al02",
    "al03",
    "al11",
    "al12",
    "al13",
    
    "ar01",
    "ar02",
    "ar03",
    "ar11",
    "ar12",
    "ar13",
    
    "polyr2",
    "polyr2len",
    "polyr2exp",
    "polyr2const",
    "polyr2start",
    
    "expl",
    "expl_start",
    "expr",
    "expr_start",
    
    "shiftc",
] 

if "TOFIT" in os.environ:
    tofit = os.environ["TOFIT"].split(",")

tofit += ["mu%i" % i for i in range(nmu)]

nparams = len(tofit) - nmu

def mc_waveform_fit(t0, *params, fit_data_only=True):
    assert(len(params) == len(tofit))
    args = {}
    for k, p in zip(tofit, params):
        if k == "mu0":
            args[k] = [p]
        elif k.startswith("mu"):
            args["mu0"].append(p)
        else:
            args[k] = p
    if "er_width" not in args:
        args["er_width"] = ER_WIDTH
        
    args["BASELINE"] = BASELINE
        
    dat = mc_waveform(t0, **args)
    if fit_data_only:
        return dat[data_fit_all]
    else:
        return dat
    
p0 = [default_params[k] for k in tofit]
bound_lo, bound_hi = list(zip(*[bounds[k] for k in tofit]))

MC_ANGLES = FIT_ANGLES

In [ ]:
def mc_waveform_optimize(params):
    return np.sum(((mc_waveform_fit(time_fitregion, *params) - data_fitregion_norm[data_fit_all]) / data_err_fitregion_norm[data_fit_all])**2)

In [ ]:
popt = optimize.differential_evolution(mc_waveform_optimize, 
                                       list(zip(bound_lo, bound_hi)), 
                                       strategy="best1bin", maxiter=100,
                                       popsize=15, tol=0.01, mutation=(0.5, 1),
                                       recombination=0.7, init="sobol",
                                       x0=p0, updating="deferred",
                                       workers=-1)

In [ ]:
# popt = optimize.basinhopping(mc_waveform_optimize, p0)

In [ ]:
# popt, _ = optimize.curve_fit(mc_waveform_fit, time_fitregion, data_fitregion_norm[data_fit_all], 
#                        sigma=data_err_fitregion_norm[data_fit_all],
#                        p0=p0, bounds=(bound_lo, bound_hi))

In [ ]:
for ip in range(len(p0)):
    print(tofit[ip], p0[ip], bound_lo[ip], bound_hi[ip], popt[ip])

In [ ]:
lcl_popt = popt

In [ ]:
#Create arrays for mc in fit and plot regions
print('pred')
pred_fitregion  = mc_waveform_fit(time_fitregion, *lcl_popt, fit_data_only=False)
pred_plotregion = mc_waveform_fit(time_plotregion, *lcl_popt, fit_data_only=False)

pred_fitregion_norm_list = [norm(v) for v in np.split(pred_fitregion, nangles)]
pred_plotregion_norm_list = [norm(v) for v in np.split(pred_plotregion, nangles)]

chi2 = calc_chi2(np.concatenate(pred_fitregion_norm_list), data_fitregion_norm, data_err_fitregion_norm)
ndf = len(np.concatenate(pred_fitregion_norm_list))

print('fit chi2/ndf =     '+str(chi2)+' / '+str(ndf))

In [ ]:
debugging = True

#Default prediction for comparison
do_fit = False
pred_default_fitregion  = mc_waveform(time_fitregion, mu0=list(popt[-nmu:]), er_width=1.3)
pred_default_plotregion = mc_waveform(time_plotregion, mu0=list(popt[-nmu:]), er_width=1.3)

pred_default_fitregion_norm_list = [norm(v) for v in np.split(pred_default_fitregion, nangles)]
pred_default_plotregion_norm_list = [norm(v) for v in np.split(pred_default_plotregion, nangles)]

chi2_default = calc_chi2(np.concatenate(pred_default_fitregion_norm_list), data_fitregion_norm, data_err_fitregion_norm)
print('default chi2/ndf = '+str(chi2_default)+' / '+str(ndf))

#Ratio of Default/Fit MC
pred_ratio_plotregion_norm = np.concatenate(pred_default_plotregion_norm_list)/np.concatenate(pred_plotregion_norm_list)
pred_ratio_plotregion_norm_list = np.split(pred_ratio_plotregion_norm, nangles)

In [ ]:
plt.close("all")

#Draw subplots
for i,a in enumerate(FIT_ANGLES):
    thlo = a
    thhi = thlo + angle_increment
    plt.figure(i+1)
    
    fig,(ax1,ax2) = plt.subplots(num=i+1,figsize=(6.4, 5.6),nrows=2, sharex=True, gridspec_kw={'height_ratios': [3, 1]})

    ax1.axhline(0, color="r")
    ax1.plot(time_plotregion_list[i], pred_default_plotregion_norm_list[i], 
             label="Nominal Response")
    ax1.plot(time_plotregion_list[i], pred_plotregion_norm_list[i], 
             label="Fit Response",color='orange')
    ax1.plot(time_plotregion_list[i], data_plotregion_norm_list[i], markersize=3,
                 linestyle="none", marker=".", color="black", label="Data")    
    
    ax1.legend(ncol=2 ,loc='upper center', bbox_to_anchor=(0.5, 1.425))
    
#     ax2.errorbar(time_plotregion_list[i], data_plotregion_norm_list[i]/pred_plotregion_norm_list[i],
#             abs(data_err_plotregion_norm_list[i]/pred_plotregion_norm_list[i]),linestyle="none", marker=".", color="black")
#     ax2.set_ylim([0.7, 1.3])
#     ax2.axhline(1, color='orange')
    
    ax2.errorbar(time_plotregion_list[i], data_plotregion_norm_list[i]-pred_default_plotregion_norm_list[i],
            abs(data_err_plotregion_norm_list[i]),linestyle="none", marker=".")
    
    ax2.errorbar(time_plotregion_list[i], data_plotregion_norm_list[i]-pred_plotregion_norm_list[i],
            abs(data_err_plotregion_norm_list[i]),linestyle="none", marker=".")

    if IPLANE == 2:
        ax2.set_ylim([-0.1, 0.1])
    else:
        ax2.set_ylim([-0.15, 0.15])

    ax2.axhline(0, color='black')
    
    #ax2.plot(time_plotregion_list[i], pred_ratio_plotregion_norm_list[i], label="Default/Fit MC Ratio")
    
    #plt.xlim([-5, 5])
    plt.subplots_adjust(hspace=0)
    ax1.text(txt_x, 0.85, "$%i^\\circ < \\theta_{xw} < %i^\\circ$" % (thlo, thhi),fontsize=16,transform=ax1.transAxes)
    ax1.text(0.05, 0.85, "Plane %i" % IPLANE, fontsize=16, transform=ax1.transAxes)
    ax2.set_xlabel("Time [$\\mu$s]")
    ax1.set_ylabel("Amp. Normalized")
    ax2.set_ylabel("Data - Resp.", fontsize=12)
    
    plt.tight_layout()
    if dosave:
        plt.savefig(savedir + "signalfit_th%ito%i.pdf" % (thlo, thhi), bbox_inches="tight")


In [ ]:
fit_param_args = dict(zip(tofit[:nparams], popt[:nparams]))

In [ ]:
# try out plotting some angles to see how the fit compares to the nominal
MC_ANGLES = np.array(list(range(0, 90, 5)))
plt_time = np.linspace(-40, 40, 800)
plt_times = np.tile(plt_time, len(MC_ANGLES))

nom_wvf = mc_waveform(plt_times, include_tail=False)
nom_wvfs = np.split(nom_wvf, len(MC_ANGLES))

fit_wvf = mc_waveform(plt_times, **fit_param_args)
fit_wvfs = np.split(fit_wvf, len(MC_ANGLES))
plt.close("all")

for i, th in enumerate(MC_ANGLES):
    plt.figure(i)
    plt.plot(plt_time, norm(nom_wvfs[i]), label="Nominal")
    plt.plot(plt_time, norm(fit_wvfs[i]), label="Fit")
    plt.title("Plane %i, $\\theta_{xw} = %i^\\circ$ Signal Response" % (IPLANE, th))
    plt.legend()
    #if th < 75:
    #    plt.xlim([-30, 30])
        
    plt.xlabel("Time [$\\mu$s]")
    plt.tight_layout()
    if dosave: plt.savefig(savedir + "fitvnom_signals_th%i.pdf" %(int(th)))

In [ ]:
plt.clf()

ER_plt = electronics_response_ideal(time, er_width)
plt.plot(time, ER_plt / ER_plt.max(), label="Nominal")

fit_er_width = fit_param_args.get("er_width", ER_WIDTH)

ER_fit = electronics_response_tail(time, fit_er_width)
if fit_param_args.get("rcrc_width", 0.) > 0:
    ER_fit = -convolve(ER_fit, RC_filter(time, fit_param_args["rcrc_width"])) + ER_fit
ER_fit = ER_fit / np.abs(ER_fit).max()

plt.plot(time, ER_fit / ER_fit.max(), label="Fit")

plt.xlim([0, 15])
plt.legend()
plt.title("Electronics Response")
plt.xlabel("Time [$\\mu$s]")
plt.tight_layout()

if dosave: plt.savefig(savedir + "fit_elecResp.pdf")

In [ ]:
sl1 = fit_param_args["sl1"]
sr1 = fit_param_args["sr1"]
sl0 = fit_param_args["sl0"]
sr0 = fit_param_args["sr0"]
weight_ratio = fit_param_args["weight_ratio"]
expl = fit_param_args.get("expl", 1)
expr = fit_param_args.get("expr", 1)
shiftc = fit_param_args.get("shiftc", 0)

nom_frac = 1

#Compuate scaled times and interpolate to get scaled paths, all for each angle
offset_param = 1-np.exp(-np.absolute(pitchpos)/1.5)  #3 mm between wires
scaling_left  = sl1*offset_param
scaling_right = sr1*offset_param
weights_left  = (sl0)*(1 - weight_ratio)
weights_right = (sr0)*1. / (1 - weight_ratio)
time_shift = offset_param*shiftc

scaled_paths = []
for i, (p_nom, p_noY) in enumerate(zip(paths_nom, paths_noY)):
    if isinstance(p_nom, tuple):
        p = (p_nom[0]*nom_frac + p_noY[0]*(1-nom_frac), 
             p_nom[1]*nom_frac + p_noY[1]*(1-nom_frac))
    else:
        p = p_nom*nom_frac + p_noY*(1-nom_frac)

    sp = scale_path(p, compute_path_center_time(p, time),
                    sl0 + scaling_left[i], sr0 + scaling_right[i],
                    weights_left, weights_right, expl, expr, time_shift[i])
    scaled_paths.append(sp)

In [ ]:
plt.clf()

for p, path in zip(pitchpos, scaled_paths):
    if isinstance(path, tuple):
        plt.plot(time, -path[0])
    else:
        plt.plot(time, -path)
    
plt.title("Plane %i Fit Electron Path Responses" % IPLANE)
plt.xlabel("Time [$\\mu$s]")
plt.ylabel("Current")

#plt.xlim([57 - 2*(2-IPLANE), 67 - 2*(2-IPLANE)])
plt.tight_layout()

# plt.axvline([time[CENTER_IND]], color="r")

if dosave: plt.savefig(savedir + "fit_pathResp.pdf", bbox_inches="tight")

In [ ]:
plt.clf()

for p, path in zip(pitchpos, scaled_paths):
    if isinstance(path, tuple):
        plt.plot(time, -path[0])
    else:
        plt.plot(time, -path)
    
plt.title("Plane %i Fit Electron Path Responses" % IPLANE)
plt.xlabel("Time [$\\mu$s]")
plt.ylabel("Current")

plt.xlim([57 - 2*(2-IPLANE), 67 - 2*(2-IPLANE)])
plt.tight_layout()

# plt.axvline([time[CENTER_IND]], color="r")

if dosave: plt.savefig(savedir + "fit_pathResp.pdf", bbox_inches="tight")

In [ ]:
for p, path in zip(pitchpos, scaled_paths):
    if p > 0:
        othr_path = scaled_paths[list(pitchpos).index(-p)]
        if isinstance(path, tuple):
            path = path[0]
            othr_path = othr_path[1]

        assert((path == othr_path).all())

In [ ]:
CENTRAL_PATHS = [100, 101, 102, 103, 104, 105]

In [ ]:
plt.clf()

for ip in CENTRAL_PATHS:
    p = pitchpos[ip]
    path = scaled_paths[ip]
    
    if isinstance(path, tuple):
        path = path[0]

    plt.plot(time, -path)
    
plt.title("Fit Electron Path Responses")
plt.xlabel("Time [$\\mu$s]")
plt.ylabel("Current")

plt.xlim([57 - 2*(2-IPLANE), 67 - 2*(2-IPLANE)])
plt.tight_layout()

In [ ]:
# Compute the normalization for the scaled paths
norm_nom = norm_v(mc_waveform(plt_time, ANGLES=np.array([0]), normalize=False, include_tail=False,
                              er_width=fit_param_args.get("er_width", 1.3)))
norm_fit = norm_v(mc_waveform(plt_time, ANGLES=np.array([0]), normalize=False, 
                              **fit_param_args))

In [ ]:
# Save paths if configured
scaled_paths_conv = []

for p in scaled_paths:
    if isinstance(p, tuple):
        p = p[0]
        
    if fit_param_args.get("rcrc_width", 0.) > 0:
        f = RC_filter(time, fit_param_args["rcrc_width"])
        p = (-convolve(p, f) + p)
        
    # normalize
    p = p * norm_nom / norm_fit
        
    scaled_paths_conv.append(p)
    
if dosave:
    for p, path in zip(pitchpos, scaled_paths_conv):
        with open(savedir + ("paths_plane%i_pitch%.1f" % (IPLANE, p)).replace(".","_") + ".txt", "w") as f:
            for e in path:
                f.write(str(e) + "\n")


In [ ]:
for ip in CENTRAL_PATHS:
    p = pitchpos[ip]
    path = scaled_paths_conv[ip]
    
    if isinstance(path, tuple):
        path = path[0]
        
    path_nom = paths_nom[ip]
    
    if isinstance(path_nom, tuple):
        path_nom = path_nom[0]

    print(p, np.sum(-path), np.sum(-path_nom))

In [ ]:
# And parameters
if dosave:
    with open(savedir + "popt.txt", "w") as f:
        for ip in range(len(p0)):
            print(tofit[ip], p0[ip], bound_lo[ip], bound_hi[ip], popt[ip], file=f)